# Transient ET model

## Model Description 
This is a transient box model with recharge and seasonal ET.

#### Dimensions: 
- 50 by 50 by 1
- dx = dy = 100 m
- dz = 100 m  

#### Subsurface Properties: 
- Homogeneous 
- K = 1.0 m/day in x and y and 0.1 m/day in z.  
- Porosity = 0.35
- Specific yield = 0.3
- Storage coefficient=0.001  

#### Boundary Conditions: 
 - Right boundary is no flow (i.e. a closed basin) 
 - Left boundary is a constant head of 70 m relative to the datum, which is located at the bottom of the domain 
 - Recharge occurs at a rate of 1E-4 m/d uniformly across the domain
 
#### ET
 - Unirrigated field with ET
     - Lower left corner (400,900) Upper Right Corner (1500,1600) (locations given as x,y)
     - Extinction depth 100 m
     - Two stressperiods: 0 ET for 30 days followed by 1e-3 ET for 335 days repeated for 16 years within the field
     - Outside this field, ET is zero for the entire time period

## 1. Setup the environment

In [ ]:
#the basics
import flopy
import numpy as np
import matplotlib as mp
import os

#additional analysis tools
import flopy.utils.binaryfile as bf
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import csv

## 2. Setup the input variables 

In [ ]:
## Discretization
nrow = 50 #number of rows
ncol = 50 #number of columns
nlay = 1  #number of layers

dx= 100 #width of grid cells in x diretion 
dy= 100 #width of grid cells in y diretion 

Lx = ncol*dx  #width of domain in x
Ly = nrow*dy #width of domain in y
ztop = 100.    #top elevation 
zbot = 0.     #bottom elevation

dz = (ztop - zbot) / nlay #cell height in z direction

## Subsurface properties
K_horiz = 1.    #assign horizontal hydraulic conductivity 
K_vert = 0.1    #assign vertical hydraulic conductivity (along z axis)
n = 0.35        #assign porosity
Ss = 0.001      #assign storage coefficient
Sy = 0.3        #assign specific yield
uncnf = 1       #0=confined, >0 = unconfined

## Recharge
recharge = 1e-4 #m/day

#ET
ET_val1 = 0 #m/day
ET_val2 = 1e-3 #m/day
ET_locxy=[[400, 900], [1500, 1600]] #lower left and upper right corners of ET in xy coordinates
extinction_depth = 100

#well - keeping this here just to have an observation point at the center of the field
pumping = 0 #m3/day  
well_xy=np.average(ET_locxy, axis=0) #xy location of well in center of ET zone
print(well_xy)
well_row = np.floor(nrow-(well_xy[1]/dy)) #convert the y location to a row
well_col=np.floor(well_xy[0]/dx) #convert the x location to a column
well_loc = (0,well_row,well_col) #Well loc should be layer, row, column
print(well_loc)
#print(well_col)


## Boundary conditions
h_left=70

### 2.1 Timing  variables 
These are the added variables for the transient simulation. 
- **nper** is the number of  model stress periods.
- The **steady** variable turns into a list with boolean entries "True/False" for each of these stress periods. The first is designated as "True" in order to set up a steady-state solution for our aquifer to use as a base for future transient stress periods. (If we just start solving transient stress periods with arbitrary starting heads, it wouldn't make for a very realistic natural groundwater system.) More stress periods are added to a simulation if there are changes to the "stress" on the aquifer, i.e. pumping, river levels, or recharge rates. (Note: some things cannot be changed per stress period, i.e. ibound, conductivity, or other properties related to the aquifer geology.)
- **perlen** designates the length of each stress period. For the steady-state stress period, it doesn't really matter what we assign here since the equation is not time-dependent. For the transient stress periods we assign 'perlen' entries that correspond to whatever we have defined our length units to be. For this model, each stress period will last 5 days.
- **nstp** is the number of steps for which MODFLOW's solver will attain solutions within the length of each stress periods. It may be adjusted depending on what you are interested in resolving in the simulation; if you wish to see high resolution changes in head profiles, a higher nstp may be used. However, this will increase the time required to solve the simulation since the solver needs to output more solutions, so if you are just interested in the general head behavior over a large number of stress periods, a lower 'nstp' may be used.

In [ ]:
# Set timing variables
sp_peryr = 2       # periods for seasonality (two stress periods per year)
sp1_len = 30       # Length of stress period 1 in days
sp2_len = 335      # Length of stress period 2 in days
sim_years = 16     # number of years for transient simulation 
nper = int(sp_peryr*sim_years + 1) # total number of stress periods the +1 is for the steady state start

#setup the steady state array
steady = np.ones(nper, dtype=bool)*False   # steady-state or transient flag for each stress period (boolean array)
steady[0] = True    # initial stress period is steady-state

#make an array of period lengths
perlen=np.tile([sp1_len, sp2_len],sim_years)
perlen=np.append(10,perlen) #add a period of length 10 at the start for the steady state simulation
print("Period lengths", perlen)

#make an nstp array for the # of time steps to solve within each period. In this case we will do daily,
#so we can repeat the perlen array
nstp=perlen*1
nstp[0]=1 # for the steady state solution we just want one output
print("Number of Steps", nstp)


## 3. Setup and run a transient MODFLOW for the ET scenario


In [ ]:
#Initialize the model
modelnameE = "ET_Model"
moddir="../../modflow/mf2005"
me = flopy.modflow.Modflow(modelnameE, exe_name = moddir)

#Discretization - dis
dis = flopy.modflow.ModflowDis(model=me, nlay=nlay, nrow=nrow, ncol=ncol, 
                               delr=dx, delc=dy, top=ztop, botm=zbot, 
                               itmuni = 4, lenuni = 2, 
                               nper=nper, steady=steady, perlen=perlen, nstp=nstp)

# Boundary and initial conditions - bas
#Define cell activity (IBOUND)
ibound = np.ones((nlay, nrow, ncol), dtype=np.int32) #integer array of dim (z,y,x), makes all cells active                                                     
ibound[:,:,0] = -1          #replace leftmost column (all zs, all ys, first x) with -1 to indicate constant head
#setup initial heads as 1 everywhere and 7 on the left boundary 
H_init = np.ones((nlay, nrow, ncol), dtype=np.float32)  #float array (i.e. decimal), sets all heads to 1.0
H_init[:, :, 0] = h_left    #replace first (left) col with desired head value
bas = flopy.modflow.ModflowBas(me, ibound=ibound, strt=H_init)

#Layer properties - lpf
lpf = flopy.modflow.ModflowLpf(me, laytyp=uncnf, hk=K_horiz, 
                               vka=K_vert, ss=Ss,
                               sy=Sy,storagecoefficient=True, ipakcb=53) 

#Recharge -
#Constant recharge throughout the simulation so I'm just repeating the recharge array for the number of stress periods
rech_zone = np.zeros((nrow,ncol))               #define an array of zeros of dim (nrow,ncol)
rech_zone=rech_zone+recharge

R = {}                    #create empty dictionary for recharge fluxes
for p in range(0,nper):   #loop over stress periods, skipping the initial steady state defined above
    R[p]=rech_zone

rch = flopy.modflow.mfrch.ModflowRch(model=me, rech=R, ipakcb=53)    #create rch object
#print(R[2])

# ET
ET_rows=(np.floor(nrow-ET_locxy[1][1]/dy),np.floor(nrow-ET_locxy[0][1]/dy)) #convert the y locations to rows
ET_cols=(np.floor(ET_locxy[0][0]/dx),np.floor(ET_locxy[1][0]/dx)) #convert the y locations to rows
ET_zone = np.zeros((nlay,nrow,ncol))     # define an array of zeroes the size of the model grid
ET_zone[0,int(ET_rows[0]):int(ET_rows[1]), int(ET_cols[0]):int(ET_cols[1])] = 1   #assign ET rate to the ET zone


#Setup alternating ET values
ET_val=np.tile([ET_val1, ET_val2],sim_years)
ET_val=np.append(ET_val1,ET_val) #add a period of length 10 at the start for the steady state simulation
                      
ET = {}
for p in range(0,nper):    #loop over stress periods
    #print(ET_val[p])
    ET[p]=ET_zone*ET_val[p]
    #print(np.sum(ET[p]))

#print(len(ET))
evt = flopy.modflow.mfevt.ModflowEvt(me, surf=ztop, evtr=ET, exdp=extinction_depth, ipakcb=53) 

#Output control - OC 
oc_spd = {}
for kper in range(nper):
    for kstp in range(nstp[kper]):
        #print(kstp)
        oc_spd[(kper, kstp)] = ['save head','save drawdown','save budget','print head','print budget']
#print(np.shape(oc_spd))
oc = flopy.modflow.ModflowOc(model=me, stress_period_data=oc_spd, compact=True)
#print('Output Control \n', oc_spd)


#Numerical solver - pcg
pcg = flopy.modflow.ModflowPcg(model=me)

#write the inputs
me.write_input()    

#Run the model 
success, mfoutput = me.run_model(pause=False, report=True)
if not success:
    raise Exception('MODFLOW did not terminate normally.')


#print(ET_rows)
#print(ET_cols)

# 4. Read in the outputs

### 4.1 For the ET Scenario

In [ ]:
hds = bf.HeadFile(modelnameE+'.hds')   #reads the binary head file
times = hds.get_times()                #returns a list of timesteps
#print(times)
#times2 = [perlen[0],perlen[0]+perlen[1],perlen[0]+perlen[1]+perlen[2]]

#print(times2)


#extract binary data from head file
head = {} #create dictionary to store head data at end of each stress period
frf = {} #create dictionary to store flows through right cell face at end of each stress period
fff = {} #create dictionary to store flows through front cell face at end of each stress period
headobj = flopy.utils.binaryfile.HeadFile(modelnameE+'.hds') #get head data as python object
budgobj = flopy.utils.binaryfile.CellBudgetFile(modelnameE+'.cbc') #get flow data as python object

#get data from python objects
for stress_per, time in enumerate(times): #iterate through times at end of each stress period
    head['sp%s'%(stress_per)] = headobj.get_data(totim=time) #append heads to head list for each stress period
    frf['sp%s'%(stress_per)] = budgobj.get_data(text='FLOW RIGHT FACE',totim=time) #append right face flow to frf list for each stress period
    fff['sp%s'%(stress_per)] = budgobj.get_data(text='FLOW FRONT FACE',totim=time) #append front face flow to fff list for each stress period


# 5. Plotting
#### 5.1 Plot the head at the center of the ET zone

In [ ]:
#plot a time series at center of field
#get time series for cell
print(well_loc)
cell_id1 = well_loc #cell at the center of the ag area
time_series1 = headobj.get_ts(cell_id1) #get the time series using flopy

#create first plot  
plt.subplot(1, 1, 1)
plt.title("Head at center of Field",fontweight='bold')
plt.xlabel('time (days)',fontweight='bold')
plt.ylabel('head (m)',fontweight='bold')
plt.plot(time_series1[:, 0], time_series1[:, 1], 'b-') #plot the time series with points at each record
plt.show()


### 5.2 Plot the head at the center of the domain

In [ ]:
#plot a time series at center of domain
#get time series for cell
print(well_loc)
cell_id2 = (0, int(nrow/2) - 1, int(ncol/2) - 1) #cell at the center of the domain
time_series2 = headobj.get_ts(cell_id2) #get the time series using flopy

#create second plot  
plt.subplot(1, 1, 1)
plt.title("Head at center of Domain")
#plt.title('Head at cell ({0},{1},{2})'.format(cell_id1[0] + 1, 
#                                              cell_id1[1] + 1, 
#                                              cell_id1[2] + 1),fontweight='bold') #create title with cell_id format
plt.xlabel('time (days)',fontweight='bold')
plt.ylabel('head (m)',fontweight='bold')
plt.plot(time_series2[:, 0], time_series2[:, 1], 'b-') #plot the time series with points at each record
plt.show()


## 6.3 Plot the head at the end of each stress period 

In [ ]:
#set X, Y, Z variables for 3d plot to be our model domain and head solution
X = np.arange(0,Lx,dx)
Y = np.arange(0,Ly,dy)
X, Y = np.meshgrid(X, Y)
times = [perlen[0],perlen[0]+perlen[1],perlen[0]+perlen[1]+perlen[2]] #extract times at end of each stress period


#create a figure for every time
for i in range(len(times)):
    #create 3d figure
    fig_3d = plt.figure(figsize=(12,5)) #create a figure instance
    ax = fig_3d.gca(projection='3d') #set an axes with 3d properties
    Z = np.flipud(head['sp%s'%i][0]) #head matrix is flipped to display properly 

    #create surface and labels
    surf = ax.plot_surface(X,Y,Z, cmap = plt.cm.coolwarm, linewidth=0, antialiased=False, label='head') #plot head surface
    fig_3d.colorbar(surf,shrink=0.5,aspect=5).set_label('Head (m)',fontsize=10,fontweight='bold') #set colorbar
    ax.set_xlabel('Lx (m)', fontsize=15, fontweight='bold')
    ax.set_ylabel('Ly (m)', fontsize=15, fontweight='bold')
    ax.set_title('Head Surface: Stress-Period %s'%(i+1), fontsize=15, fontweight='bold')
    plt.show(surf)